In [15]:
import zipfile
import csv
import pandas as pd
import os
import logging
from os.path import exists

In [16]:
if not os.path.exists('exported'):
    os.makedirs('exported')

In [17]:
#Logging File for Metrics
logging.basicConfig(filename='logs.log', level=logging.ERROR)

#Text File for Unit Tests
UnitTest = open(r"C:\Users\KEVAL\Desktop\CapstoneUnitTesting\DataEngineering_CapstoneProject\UnitTest.txt", "w")
UnitTest.write("This is the UNIT Test File"+ "\n")

27

In [18]:
#Check if file is in folder

def filecheck(location):
    # check if it exist
    try:
        file_exists = os.path.exists(location)
        path = os.path.dirname(os.path.abspath(location))
        #Print to UnitTest
        UnitTest.write("Successfull!! -- The path of the file is: " + path + "-->" + location + "\n")
    except Exception as e:
        #log Error into text file as e
        #Print to UnitTest.txt
        UnitTest.write("ERROR: Unsuccessfull!! -- The path of the file is: " + path + "-->" + location + "\n")
        print(e)
        print("Error: File NOT IN LOCATION " + location)
        return
    return 

Download data from source and store locally. The file is then inzipped so the data can be viewed. 

In [19]:
!kaggle datasets download --force -d felix4guti/traffic-violations-in-usa
print('downloaded data')
with zipfile.ZipFile("./{}.zip".format("traffic-violations-in-usa"),"r") as z:
    z.extractall(".")
print('unzipped the data')


downloaded data



  0%|          | 0.00/48.5M [00:00<?, ?B/s]
  2%|2         | 1.00M/48.5M [00:00<00:06, 7.94MB/s]
  4%|4         | 2.00M/48.5M [00:00<00:07, 6.77MB/s]
  8%|8         | 4.00M/48.5M [00:00<00:04, 11.3MB/s]
 12%|#2        | 6.00M/48.5M [00:00<00:03, 14.0MB/s]
 16%|#6        | 8.00M/48.5M [00:00<00:02, 16.1MB/s]
 23%|##2       | 11.0M/48.5M [00:00<00:02, 18.1MB/s]
 27%|##6       | 13.0M/48.5M [00:00<00:02, 17.7MB/s]
 31%|###       | 15.0M/48.5M [00:04<00:17, 2.03MB/s]
 35%|###5      | 17.0M/48.5M [00:05<00:17, 1.93MB/s]
 37%|###7      | 18.0M/48.5M [00:05<00:14, 2.14MB/s]
 39%|###9      | 19.0M/48.5M [00:05<00:12, 2.50MB/s]
 41%|####1     | 20.0M/48.5M [00:05<00:10, 2.95MB/s]
 43%|####3     | 21.0M/48.5M [00:05<00:08, 3.49MB/s]
 45%|####5     | 22.0M/48.5M [00:06<00:06, 4.09MB/s]
 47%|####7     | 23.0M/48.5M [00:06<00:05, 4.71MB/s]
 49%|####9     | 24.0M/48.5M [00:06<00:04, 5.39MB/s]
 54%|#####3    | 26.0M/48.5M [00:06<00:03, 7.14MB/s]
 58%|#####7    | 28.0M/48.5M [00:06<00:02, 8.95MB/s]
 

unzipped the data


The dataset is opened. 

In [20]:
class Dataset:
    def __init__(self, csv_filename):
        self.dataset = pd.read_csv(csv_filename, dtype='unicode')
        filecheck(csv_filename)
    
    def __generateMetrics(self, subset, columns):
        numberOfRows = len(subset)
        numberOfColumns = len(columns)
        output = "Number of Rows: {}\n".format(numberOfRows) + "Number of Columns: {}\n".format(numberOfColumns) 
        return output
        
    def generate_subset(self, table_name, columns):
        test_data = self.dataset[columns]
        try:
            path = os.path.join('exported', table_name)
            test_data.to_csv(path+'.csv')
            #Check if csv file was made
            filecheck(path+'csv')
            with open(path+'_metrics.txt', 'w') as file:
                file.write(self.__generateMetrics(test_data, columns))
                #Check if the metrics file was made
                filecheck(path+'_metrics.txt')
        except Exception as e:
            logging.error(e)
            print(e)
            return
        return test_data.head()

In [21]:
traffic_data = Dataset("Traffic_Violations.csv")

In [22]:
traffic_data.generate_subset('Incident', ['Arrest Type', 'Description', 'State', 'Violation Type', 'Charge', 'Article', 'Contributed To Accident'])

,Arrest Type,Description,State,Violation Type,Charge,Article,Contributed To Accident
0,A - Marked Patrol,DRIVING VEHICLE ON HIGHWAY WITH SUSPENDED REGI...,MD,Citation,13-401(h),Transportation Article,No
1,A - Marked Patrol,DRIVING WHILE IMPAIRED BY ALCOHOL,MD,Citation,21-902(b1),Transportation Article,No
2,A - Marked Patrol,FAILURE TO STOP AT STOP SIGN,MD,Citation,21-707(a),Transportation Article,No
3,A - Marked Patrol,DRIVER USING HANDS TO USE HANDHELD TELEPHONE W...,VA,Citation,21-1124.2(d2),Transportation Article,No
4,A - Marked Patrol,FAILURE STOP AND YIELD AT THRU HWY,MD,Citation,21-403(b),Transportation Article,No


In [23]:
traffic_data.generate_subset('Report', ['HAZMAT', 'Commercial License', 'Fatal', 'Property Damage', 'Personal Injury', 'Belts', 'Accident'])

,HAZMAT,Commercial License,Fatal,Property Damage,Personal Injury,Belts,Accident
0,No,No,No,No,No,No,No
1,No,No,No,No,No,No,No
2,No,No,No,No,No,No,No
3,No,No,No,No,No,No,No
4,No,No,No,Yes,No,No,No


In [24]:
traffic_data.generate_subset('Time', ['Date Of Stop', 'Time Of Stop', 'Location', 'Latitude', 'Longitude', 'Geolocation'])

,Date Of Stop,Time Of Stop,Location,Latitude,Longitude,Geolocation
0,09/24/2013,17:11:00,8804 FLOWER AVE,NaN,NaN,NaN
1,12/20/2012,00:41:00,NORFOLK AVE / ST ELMO AVE,38.9835782,-77.09310515,"(38.9835782, -77.09310515)"
2,07/20/2012,23:12:00,WISTERIA DR @ WARING STATION RD,39.1618098166667,-77.25358095,"(39.1618098166667, -77.25358095)"
3,03/19/2012,16:10:00,CLARENDON RD @ ELM ST. N/,38.9827307333333,-77.1007551666667,"(38.9827307333333, -77.1007551666667)"
4,12/01/2014,12:52:00,CHRISTOPHER AVE/MONTGOMERY VILLAGE AVE,39.1628883333333,-77.2290883333333,"(39.1628883333333, -77.2290883333333)"


In [25]:
traffic_data.generate_subset('Agency', ['Agency'])

,Agency
0,MCP
1,MCP
2,MCP
3,MCP
4,MCP


In [26]:
traffic_data.generate_subset('Subagency', ['SubAgency'])

,SubAgency
0,"3rd district, Silver Spring"
1,"2nd district, Bethesda"
2,"5th district, Germantown"
3,"2nd district, Bethesda"
4,"6th district, Gaithersburg / Montgomery Village"


In [27]:
traffic_data.generate_subset('Driver', ['DL State', 'Driver State', 'Driver City', 'Gender', 'Race'])

,DL State,Driver State,Driver City,Gender,Race
0,MD,MD,TAKOMA PARK,M,BLACK
1,MD,MD,DERWOOD,M,WHITE
2,MD,MD,GERMANTOWN,F,ASIAN
3,VA,VA,ARLINGTON,M,HISPANIC
4,MD,MD,UPPER MARLBORO,F,BLACK


In [28]:
traffic_data.generate_subset('Vehicle', ['VehicleType', 'Make', 'Year', 'Model', 'Color'])

,VehicleType,Make,Year,Model,Color
0,02 - Automobile,FORD,2008,4S,BLACK
1,02 - Automobile,AUDI,2005,4S,GRAY
2,02 - Automobile,TOYT,2002,4S,RED
3,02 - Automobile,HONDA,1996,CIVIC,SILVER
4,02 - Automobile,HONDA,2001,ACCORD,SILVER
